### Training the XGBoost model with HyperParameter Tuning

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import mlflow

# Setup MLFlow Tracking Server
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow.set_tracking_uri(mlflow_tracking_uri)

e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [3]:
# Set or Create an experiment
mlflow.set_experiment("Exp 5- ML Algorithms with Hyperparameter Tuning")

2025/12/26 20:15:03 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5- ML Algorithms with Hyperparameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/4b2734eb45ec4dac9b84b612a9b1df2d', creation_time=1766760302303, experiment_id='7', last_update_time=1766760302303, lifecycle_stage='active', name='Exp 5- ML Algorithms with Hyperparameter Tuning', tags={}>

In [5]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [6]:
df = pd.read_csv('./artifacts/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
print(df.shape)
df.sample(5)

(36662, 2)


,clean_comment,category
4723,randians heart ache bangladeshi hmm,0
30725,demonetisation usurping cbi dismantling power ...,-1
35533,communist understood economics communist,0
28460,indiadiscussion best place,1
15818,commenting later,0


In [7]:
import os

ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

In [8]:
# For XGBoost, remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Remove the rows where target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)
max_features = 1000

# Split the data into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)


In [9]:
# Function to log results in MLFlow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run() as run:
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "Algorithm_Comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("Algo_name", model_name)

        # Train the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log Accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("Accuracy: ", accuracy)

        # Log Classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    if metric != "support":
                        mlflow.log_metric(f"{label}_{metric}", value)

        # Log Confusion Matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Model={model_name}")

        # Save and log the confusion matrix plot
        filename = f"Exp5_TF_IDF_Trigrams_{model_name}_ConfusionMatrix.png"
        cm_path = os.path.join(ARTIFACT_DIR, filename)
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_Model_TF_IDF_Trigrams_SMOTE")


In [10]:
# Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))

In [11]:
# Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLFLow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

In [12]:
# Run the experiment for XGBoost
run_optuna_experiment()

[I 2025-12-26 20:26:33,368] A new study created in memory with name: no-name-be9c3eec-3053-482d-a1c0-7732691e36b7
[I 2025-12-26 20:28:11,240] Trial 0 finished with value: 0.6833492431474157 and parameters: {'n_estimators': 202, 'learning_rate': 0.021324888677062535, 'max_depth': 6}. Best is trial 0 with value: 0.6833492431474157.
[I 2025-12-26 20:29:04,628] Trial 1 finished with value: 0.597708986772126 and parameters: {'n_estimators': 133, 'learning_rate': 0.008863435982251387, 'max_depth': 5}. Best is trial 0 with value: 0.6833492431474157.
[I 2025-12-26 20:29:55,368] Trial 2 finished with value: 0.5548888585844811 and parameters: {'n_estimators': 74, 'learning_rate': 0.0014690656527836929, 'max_depth': 7}. Best is trial 0 with value: 0.6833492431474157.
[I 2025-12-26 20:31:25,241] Trial 3 finished with value: 0.6170735033410609 and parameters: {'n_estimators': 97, 'learning_rate': 0.002946658400915531, 'max_depth': 10}. Best is trial 0 with value: 0.6833492431474157.
[I 2025-12-26 2